<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [11]:
## Task 1: Mark all launch sites on a map
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

# Create a DataFrame that contains the launch site data
launch_sites_df = pd.DataFrame({
    'Site': ['Kennedy Space Center', 'Cape Canaveral', 'Vandenberg SFB', 'Baikonur Cosmodrome'],
    'Lat': [28.5729, 28.4894, 34.7420, 45.9646],
    'Long': [-80.6490, -80.5769, -120.5724, 63.3052]
})

launch_sites_map = folium.Map(location=[0, 0], zoom_start=2)

marker_cluster = MarkerCluster().add_to(launch_sites_map)


for idx, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Site'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(marker_cluster)

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='bottomleft',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter
)
launch_sites_map.add_child(mouse_position)

legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 120px; height: 90px; 
    border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
    ">&nbsp; Launch Sites <br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i> 
    </div>
'''
launch_sites_map.get_root().html.add_child(folium.Element(legend_html))

launch_sites_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [14]:
# Download and read the `spacex_launch_geo.csv`
import folium
import pandas as pd

launch_sites_df = pd.DataFrame({
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Latitude': [28.5618571, 28.5618571, 28.6080585, 34.632093],
    'Longitude': [-80.577366, -80.577366, -80.6039558, -120.610829]
})

launch_sites_map = folium.Map(location=[28.5618571, -80.577366], zoom_start=5)

for idx, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(launch_sites_map)

launch_sites_map

Now, you can take a look at what are the coordinates for each site.


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [ ]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle and marker for NASA Johnson Space Center
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values
# In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Add circle
    circle = folium.Circle(
        coordinate, 
        radius=1000, 
        color='#000000', 
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))
    site_map.add_child(circle)
    
    # Add marker
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    )
    site_map.add_child(marker)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [15]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster

launch_data = pd.DataFrame({
    'Launch Site': ['CCAFS LC-40', 'CCAFS LC-40', 'KSC LC-39A', 'VAFB SLC-4E', 'CCAFS SLC-40'],
    'Lat': [28.5618571, 28.5618571, 28.6080585, 34.632093, 28.5618571],
    'Long': [-80.577366, -80.577366, -80.6039558, -120.610829, -80.577366],
    'Class': [1, 0, 1, 1, 0]  # 1表示成功，0表示失败
})

site_map = folium.Map(location=[30, -90], zoom_start=4)

marker_cluster = MarkerCluster().add_to(site_map)


for idx, launch in launch_data.iterrows():
    
    color = 'green' if launch['Class'] == 1 else 'red'
    
    folium.Marker(
        location=[launch['Lat'], launch['Long']],
        popup=f"Site: {launch['Launch Site']}, {'Success' if launch['Class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(marker_cluster)

site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [18]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster

spacex_df = pd.DataFrame({
    'LaunchSite': ['CCAFS LC-40', 'CCAFS LC-40', 'KSC LC-39A', 'VAFB SLC-4E', 'CCAFS SLC-40'] * 20,
    'Latitude': [28.5618571, 28.5618571, 28.6080585, 34.632093, 28.5618571] * 20,
    'Longitude': [-80.577366, -80.577366, -80.6039558, -120.610829, -80.577366] * 20,
    'Class': [1, 0, 1, 1, 0] * 20  
})

site_success = spacex_df.groupby('LaunchSite')['Class'].mean().reset_index()
site_success = site_success.rename(columns={'Class': 'SuccessRate'})

site_coords = spacex_df.groupby('LaunchSite').agg({
    'Latitude': 'first',
    'Longitude': 'first'
}).reset_index()

site_info = pd.merge(site_success, site_coords, on='LaunchSite')

site_map = folium.Map(location=[30, -90], zoom_start=4)

marker_cluster = MarkerCluster().add_to(site_map)

for idx, site in site_info.iterrows():
    
    radius = site['SuccessRate'] * 15000 
    
    
    if site['SuccessRate'] >= 0.8:
        color = 'green'
    elif site['SuccessRate'] >= 0.6:
        color = 'orange'
    else:
        color = 'red'
    
    
    folium.CircleMarker(
        location=[site['Latitude'], site['Longitude']],
        radius=radius,
        popup=f"Site: {site['LaunchSite']}<br>Success Rate: {site['SuccessRate']:.2%}",
        color=color,
        fill=True,
        fillColor=color
    ).add_to(marker_cluster)


legend_html = """
    <div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: 90px; 
    border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
    ">
        &nbsp; Launch Site Success Rate <br>
        &nbsp; <i class="fa fa-circle fa-1x" style="color:green"></i> &ge; 80% <br>
        &nbsp; <i class="fa fa-circle fa-1x" style="color:orange"></i> 60% - 80% <br>
        &nbsp; <i class="fa fa-circle fa-1x" style="color:red"></i> &lt; 60%
    </div>
"""
site_map.get_root().html.add_child(folium.Element(legend_html))


site_map

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:
# Define a function to assign color based on the class value
def assign_marker_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to create a new 'marker_color' column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Display the first few rows of the updated dataframe to verify the new column
print(spacex_df[['Launch Site', 'class', 'marker_color']].head())

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster
for index, record in spacex_df.iterrows():
    # Convert to float to avoid wrong data type
    lat = float(record['Lat'])
    long = float(record['Long'])
    label = f"{record['Launch Site']}: {'Success' if record['class'] == 1 else 'Failure'}"
    
    # Create and add a Marker for each launch record
    marker = folium.Marker(
        [lat, long],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=label
    )
    marker_cluster.add_child(marker)

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Create a legend
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 220px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     ">&nbsp; Launch Success/Failure <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Success &nbsp;
     <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; Failure
</div>
'''
site_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [19]:
import numpy as np
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    r = 6371  
    
    return c * r


launch_site = {
    'name': 'CCAFS LC-40',
    'lat': 28.5618571,
    'lon': -80.577366
}


proximities = [
    {'name': 'Orlando', 'lat': 28.5383, 'lon': -81.3792},
    {'name': 'Tampa', 'lat': 27.9506, 'lon': -82.4572},
    {'name': 'Miami', 'lat': 25.7617, 'lon': -80.1918},
    {'name': 'Jacksonville', 'lat': 30.3322, 'lon': -81.6557},
    {'name': 'Atlanta', 'lat': 33.7490, 'lon': -84.3880}
]


distances = []
for proximity in proximities:
    distance = calculate_distance(
        launch_site['lat'], launch_site['lon'],
        proximity['lat'], proximity['lon']
    )
    distances.append({
        'Launch Site': launch_site['name'],
        'Proximity': proximity['name'],
        'Distance (km)': round(distance, 2)
    })

distances_df = pd.DataFrame(distances)
distances_df = distances_df.sort_values('Distance (km)')

print(f"Distances from {launch_site['name']} to nearby cities:")
print(distances_df)

nearest_city = distances_df.iloc[0]
farthest_city = distances_df.iloc[-1]

print(f"\nNearest city: {nearest_city['Proximity']} ({nearest_city['Distance (km)']} km)")
print(f"Farthest city: {farthest_city['Proximity']} ({farthest_city['Distance (km)']} km)")

Distances from CCAFS LC-40 to nearby cities:
   Launch Site     Proximity  Distance (km)
0  CCAFS LC-40       Orlando          78.36
1  CCAFS LC-40         Tampa         196.26
3  CCAFS LC-40  Jacksonville         222.83
2  CCAFS LC-40         Miami         313.69
4  CCAFS LC-40       Atlanta         681.15

Nearest city: Orlando (78.36 km)
Farthest city: Atlanta (681.15 km)


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
import folium
from folium.plugins import MousePosition
import pandas as pd

spacex_df = pd.DataFrame({
    'LaunchSite': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Latitude': [28.5618571, 28.5618571, 28.6080585, 34.632093],
    'Longitude': [-80.577366, -80.577366, -80.6039558, -120.610829]
})

site_map = folium.Map(location=[30, -90], zoom_start=4)

for idx, site in spacex_df.iterrows():
    folium.Marker(
        location=[site['Latitude'], site['Longitude']],
        popup=site['LaunchSite'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(site_map)
    
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=5,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter
)

site_map.add_child(mouse_position)

site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
from folium.plugins import MousePosition
from math import sin, cos, sqrt, atan2, radians

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Add markers for each launch site
for index, record in spacex_df.iterrows():
    lat = float(record['Lat'])
    long = float(record['Long'])
    label = f"{record['Launch Site']}: {'Success' if record['class'] == 1 else 'Failure'}"
    
    marker = folium.Marker(
        [lat, long],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=label
    )
    marker_cluster.add_child(marker)

# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Define the distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Function to find the closest coastline coordinate
def find_closest_coastline(launch_site_lat, launch_site_lon):
    # Example coastline coordinates (you should replace these with actual data)
    coastlines = [
        (28.56367, -80.57163),
        (28.57367, -80.58163),
        (28.55367, -80.56163),
    ]
    
    closest_distance = float('inf')
    closest_coordinate = None
    
    for coast_lat, coast_lon in coastlines:
        distance = calculate_distance(launch_site_lat, launch_site_lon, coast_lat, coast_lon)
        if distance < closest_distance:
            closest_distance = distance
            closest_coordinate = (coast_lat, coast_lon)
    
    return closest_coordinate, closest_distance

# Let the user choose a launch site
launch_sites = spacex_df['Launch Site'].unique().tolist()
print("Available launch sites:")
for i, site in enumerate(launch_sites, 1):
    print(f"{i}. {site}")

choice = int(input("Choose a launch site (enter the number): ")) - 1
selected_site = launch_sites[choice]

# Get coordinates of the selected launch site
site_data = spacex_df[spacex_df['Launch Site'] == selected_site].iloc[0]
launch_site_lat = site_data['Lat']
launch_site_lon = site_data['Long']

# Find the closest coastline
closest_coastline, distance_coastline = find_closest_coastline(launch_site_lat, launch_site_lon)

print(f"\nSelected Launch Site: {selected_site}")
print(f"Launch Site Coordinates: Lat: {launch_site_lat}, Lon: {launch_site_lon}")
print(f"Closest Coastline Coordinates: Lat: {closest_coastline[0]}, Lon: {closest_coastline[1]}")
print(f"Distance to closest coastline: {distance_coastline:.2f} km")

# Display the map
site_map

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )


from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Add markers for each launch site
for index, record in spacex_df.iterrows():
    lat = float(record['Lat'])
    long = float(record['Long'])
    label = f"{record['Launch Site']}: {'Success' if record['class'] == 1 else 'Failure'}"
    
    marker = folium.Marker(
        [lat, long],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=label
    )
    marker_cluster.add_child(marker)

# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Define the distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Function to find the closest coastline coordinate
def find_closest_coastline(launch_site_lat, launch_site_lon):
    # Example coastline coordinates (you should replace these with actual data)
    coastlines = [
        (28.56367, -80.57163),
        (28.57367, -80.58163),
        (28.55367, -80.56163),
    ]
    
    closest_distance = float('inf')
    closest_coordinate = None
    
    for coast_lat, coast_lon in coastlines:
        distance = calculate_distance(launch_site_lat, launch_site_lon, coast_lat, coast_lon)
        if distance < closest_distance:
            closest_distance = distance
            closest_coordinate = (coast_lat, coast_lon)
    
    return closest_coordinate, closest_distance

# Let the user choose a launch site
launch_sites = spacex_df['Launch Site'].unique().tolist()
print("Available launch sites:")
for i, site in enumerate(launch_sites, 1):
    print(f"{i}. {site}")

choice = int(input("Choose a launch site (enter the number): ")) - 1
selected_site = launch_sites[choice]

# Get coordinates of the selected launch site
site_data = spacex_df[spacex_df['Launch Site'] == selected_site].iloc[0]
launch_site_lat = site_data['Lat']
launch_site_lon = site_data['Long']

# Find the closest coastline
closest_coastline, distance_coastline = find_closest_coastline(launch_site_lat, launch_site_lon)

# Add a marker for the selected launch site
launch_site_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    icon=folium.Icon(color='red', icon='info-sign'),
    popup=f"Launch Site: {selected_site}"
)
site_map.add_child(launch_site_marker)

# Add a marker for the closest coastline point
coastline_marker = folium.Marker(
    closest_coastline,
    icon=folium.Icon(color='blue', icon='info-sign'),
    popup="Closest Coastline Point"
)
site_map.add_child(coastline_marker)

# Add a marker to display the distance
midpoint = [(launch_site_lat + closest_coastline[0])/2, (launch_site_lon + closest_coastline[1])/2]
distance_marker = folium.Marker(
    midpoint,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

# Draw a line between the launch site and coastline point
folium.PolyLine(locations=[
    [launch_site_lat, launch_site_lon],
    closest_coastline
], color="green", weight=2, opacity=0.8).add_to(site_map)

# Fit the map to show all markers
site_map.fit_bounds(site_map.get_bounds())

# Display the map
site_map

# Print additional information
print(f"\nSelected Launch Site: {selected_site}")
print(f"Launch Site Coordinates: Lat: {launch_site_lat}, Lon: {launch_site_lon}")
print(f"Closest Coastline Coordinates: Lat: {closest_coastline[0]}, Lon: {closest_coastline[1]}")
print(f"Distance to closest coastline: {distance_coastline:.2f} km")

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Add markers for each launch site
for index, record in spacex_df.iterrows():
    lat = float(record['Lat'])
    long = float(record['Long'])
    label = f"{record['Launch Site']}: {'Success' if record['class'] == 1 else 'Failure'}"
    
    marker = folium.Marker(
        [lat, long],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=label
    )
    marker_cluster.add_child(marker)

# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Define the distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Function to find the closest coastline coordinate
def find_closest_coastline(launch_site_lat, launch_site_lon):
    # Example coastline coordinates (you should replace these with actual data)
    coastlines = [
        (28.56367, -80.57163),
        (28.57367, -80.58163),
        (28.55367, -80.56163),
    ]
    
    closest_distance = float('inf')
    closest_coordinate = None
    
    for coast_lat, coast_lon in coastlines:
        distance = calculate_distance(launch_site_lat, launch_site_lon, coast_lat, coast_lon)
        if distance < closest_distance:
            closest_distance = distance
            closest_coordinate = (coast_lat, coast_lon)
    
    return closest_coordinate, closest_distance

# Let the user choose a launch site
launch_sites = spacex_df['Launch Site'].unique().tolist()
print("Available launch sites:")
for i, site in enumerate(launch_sites, 1):
    print(f"{i}. {site}")

choice = int(input("Choose a launch site (enter the number): ")) - 1
selected_site = launch_sites[choice]

# Get coordinates of the selected launch site
site_data = spacex_df[spacex_df['Launch Site'] == selected_site].iloc[0]
launch_site_lat = site_data['Lat']
launch_site_lon = site_data['Long']

# Find the closest coastline
closest_coastline, distance_coastline = find_closest_coastline(launch_site_lat, launch_site_lon)

# Add a marker for the selected launch site
launch_site_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    icon=folium.Icon(color='red', icon='info-sign'),
    popup=f"Launch Site: {selected_site}"
)
site_map.add_child(launch_site_marker)

# Add a marker for the closest coastline point
coastline_marker = folium.Marker(
    closest_coastline,
    icon=folium.Icon(color='blue', icon='info-sign'),
    popup="Closest Coastline Point"
)
site_map.add_child(coastline_marker)

# Add a marker to display the distance
midpoint = [(launch_site_lat + closest_coastline[0])/2, (launch_site_lon + closest_coastline[1])/2]
distance_marker = folium.Marker(
    midpoint,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

# Draw a PolyLine between the launch site and the closest coastline point
coordinates = [
    [launch_site_lat, launch_site_lon],
    closest_coastline
]
lines = folium.PolyLine(locations=coordinates, weight=2, color='red', opacity=0.8)
site_map.add_child(lines)

# Fit the map to show all markers
site_map.fit_bounds(site_map.get_bounds())

# Display the map
site_map

# Print additional information
print(f"\nSelected Launch Site: {selected_site}")
print(f"Launch Site Coordinates: Lat: {launch_site_lat}, Lon: {launch_site_lon}")
print(f"Closest Coastline Coordinates: Lat: {closest_coastline[0]}, Lon: {closest_coastline[1]}")
print(f"Distance to closest coastline: {distance_coastline:.2f} km")

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
import folium
from folium.plugins import MousePosition, MarkerCluster
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Add markers for each launch site
for index, record in spacex_df.iterrows():
    lat = float(record['Lat'])
    long = float(record['Long'])
    label = f"{record['Launch Site']}: {'Success' if record['class'] == 1 else 'Failure'}"
    
    marker = folium.Marker(
        [lat, long],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=label
    )
    marker_cluster.add_child(marker)

# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Define the distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Function to find the closest point of interest
def find_closest_point(launch_site_lat, launch_site_lon, points):
    closest_distance = float('inf')
    closest_point = None
    
    for name, (lat, lon) in points.items():
        distance = calculate_distance(launch_site_lat, launch_site_lon, lat, lon)
        if distance < closest_distance:
            closest_distance = distance
            closest_point = (name, (lat, lon))
    
    return closest_point, closest_distance

# Let the user choose a launch site
launch_sites = spacex_df['Launch Site'].unique().tolist()
print("Available launch sites:")
for i, site in enumerate(launch_sites, 1):
    print(f"{i}. {site}")

choice = int(input("Choose a launch site (enter the number): ")) - 1
selected_site = launch_sites[choice]

# Get coordinates of the selected launch site
site_data = spacex_df[spacex_df['Launch Site'] == selected_site].iloc[0]
launch_site_lat = site_data['Lat']
launch_site_lon = site_data['Long']

# Define points of interest (replace with actual data)
points_of_interest = {
    "City": (28.3922, -80.6077),  # Titusville, FL
    "Railway": (28.5722, -80.5772),
    "Highway": (28.5638, -80.5777),
    "Coastline": (28.5728, -80.5668)
}

# Add a marker for the selected launch site
launch_site_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    icon=folium.Icon(color='red', icon='info-sign'),
    popup=f"Launch Site: {selected_site}"
)
site_map.add_child(launch_site_marker)

# Colors for different types of points
color_map = {
    "City": "purple",
    "Railway": "black",
    "Highway": "orange",
    "Coastline": "blue"
}

# Find and mark the closest point for each type
for point_type, coordinates in points_of_interest.items():
    closest_point, distance = find_closest_point(launch_site_lat, launch_site_lon, {point_type: coordinates})
    
    # Add marker for the point of interest
    poi_marker = folium.Marker(
        coordinates,
        icon=folium.Icon(color=color_map[point_type], icon='info-sign'),
        popup=f"Closest {point_type}"
    )
    site_map.add_child(poi_marker)
    
    # Add a marker to display the distance
    midpoint = [(launch_site_lat + coordinates[0])/2, (launch_site_lon + coordinates[1])/2]
    distance_marker = folium.Marker(
        midpoint,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:{color_map[point_type]};"><b>{distance:.2f} KM</b></div>'
        )
    )
    site_map.add_child(distance_marker)
    
    # Draw a PolyLine between the launch site and the point of interest
    line_coordinates = [
        [launch_site_lat, launch_site_lon],
        coordinates
    ]
    line = folium.PolyLine(locations=line_coordinates, weight=2, color=color_map[point_type], opacity=0.8)
    site_map.add_child(line)
    
    print(f"Distance to closest {point_type}: {distance:.2f} km")

# Fit the map to show all markers
site_map.fit_bounds(site_map.get_bounds())

# Display the map
site_map

print(f"\nSelected Launch Site: {selected_site}")
print(f"Launch Site Coordinates: Lat: {launch_site_lat}, Lon: {launch_site_lon}")

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
